In [1]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

In [2]:
traindf = pd.read_csv('data/fixed_train.csv')
validdf = pd.read_csv('data/fixed_valid.csv')
testdf = pd.read_csv('data/fixed_test.csv')

In [3]:
# replace "_comma_" with ","
def replace_comma(df):
    df.loc[:, 'prompt'] = df['prompt'].str.replace('_comma_', ',')
    df.loc[:, 'utterance'] = df['utterance'].str.replace('_comma_', ',')

replace_comma(traindf)
replace_comma(validdf)
replace_comma(testdf)

In [4]:
def concate_conv(df):
    df_concat = pd.DataFrame(columns=df.columns) \
                .drop(['utterance_idx'], axis=1) \
                .rename(columns={'utterance': 'conv'})

    conv_id_groups = df.groupby(['conv_id'])

    for _, indices in tqdm(conv_id_groups.groups.items()):
        conv_rows = df.loc[indices].copy().sort_values(['utterance_idx'])
        conv_row = conv_rows.iloc[0].drop(['utterance_idx']).rename({'utterance': 'conv'})
        concat_text = \
            ' '.join([(row['utterance'] ) for _, row in conv_rows.iterrows()])
        conv_row['conv'] = concat_text
        df_concat = df_concat.append(conv_row, ignore_index=True)
        
    return df_concat

new_traindf = concate_conv(traindf)
new_validdf = concate_conv(validdf)
new_testdf = concate_conv(testdf)

In [6]:
print('train:', new_traindf["conv"].values[1])
print('valid:', new_validdf["conv"].values[1])
print('test:', new_testdf["conv"].values[1])

train: My girlfriend got me a pet toad today! Do you like toads? I do! I was so happy when I opened the box and that fat mofo jumped out! That was nice of your girlfriend, do you love her?
valid: Im expecting a good bonus to be on this check coming up. I can finally go buy a new car! That is some exciting news. Do you already know what kind of vehicle you want? Yes! Very exciting! Yes I had my eye on one all year. I cant wait Christmas is coming early. You will have to take me for a ride.
test: I got something nice the other day, chocolates from my husband Wow that must have been a surprise for you It was, and he does that type of thing a lot. you got a great husband


In [6]:
new_traindf.to_csv('data/new_train_0608.csv', index=False, encoding='utf8')
new_validdf.to_csv('data/new_valid_0608.csv', index=False, encoding='utf8')
new_testdf.to_csv('data/new_test_0608.csv', index=False, encoding='utf8')